In [ ]:
# for dev purpose - get username
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [ ]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import numpy as np
import datetime as dt
import time
import warnings
from snp_query_box import DsnpHelperFunction, snpQueries
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating dsnp_base_member_current_active_df in the shared storage")

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
last_month = last_date_of_last_month_str[:7]

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
# pull dsnp plan dand state maps
from snp_query_box import dsnpVal 
dsnp_plan_list = dsnpVal.dsnp_plan_list
print(dsnp_plan_list[:3])
dsnp_state_map_df = DsnpHelperFunction.get_dsnp_contract_pbp_state_mapping_file()
dsnp_state_map_df.head(1)

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
# locate data dump folder
if new_pull:
    pull_snp_member = snpQueries.pull_snp_member_bq(enroll_start_date = today_str, enroll_end_date = today_str)
    pull_snp_member['kpbp'] = pull_snp_member['Contract_Number'] + '-' + pull_snp_member['PBP']

    #TODO 2025 CSNP is added, dsnp_plan_list
    dsnp_base_member_df = pull_snp_member.loc[(pull_snp_member['kpbp'].isin(dsnp_plan_list)) | \
                                              pull_snp_member['SNP'].isin(['Y', 'D', 'F', 'I', 'C'])]
    dsnp_base_member_df = dsnp_base_member_df.merge(dsnp_state_map_df, how='left', on=['Contract_Number', 'PBP'])
    if dsnp_base_member_df['Dual_Status'].nunique()<5:
        raise ValueError("Dual_status column is missing")

    dsnp_base_member_df.to_parquet(f"{output_path}/dsnp_base_member_current_active_df.parquet")

else:
    try:
        dsnp_base_member_df = pd.read_parquet(f"{output_path}/dsnp_base_member_current_active_df.parquet")

    except:
        pull_snp_member = snpQueries.pull_snp_member_bq(enroll_start_date = today_str, enroll_end_date = today_str)
        pull_snp_member['kpbp'] = pull_snp_member['Contract_Number'] + '-' + pull_snp_member['PBP']
    
        dsnp_base_member_df = pull_snp_member.loc[(pull_snp_member['kpbp'].isin(dsnp_plan_list)) | \
                                                  pull_snp_member['SNP'].isin(['Y', 'D', 'F', 'I', 'C'])]
        dsnp_base_member_df = dsnp_base_member_df.merge(dsnp_state_map_df, how='left', on=['Contract_Number', 'PBP'])
        if dsnp_base_member_df['Dual_Status'].nunique()<5:
            raise ValueError("Dual_status column is missing")

        dsnp_base_member_df.to_parquet(f"{output_path}/dsnp_base_member_current_active_df.parquet")

In [ ]:
dsnp_base_member_df['SNP'].value_counts()

In [ ]:
# output the top rows and size
print(dsnp_base_member_df.shape)
print(dsnp_base_member_df.head(3))

In [ ]:
dsnp_base_member_df['Dual_Status'].value_counts()

In [ ]:
dsnp_base_member_df.head()

In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))